In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
from datetime import datetime
import selenium
from selenium import webdriver

In [2]:
# Action Network URL.
nba_url = 'https://www.actionnetwork.com/nba/odds'

# Takes the URL and makes it an html - takes in a string (URL) and returns
# the html in string form.
def get_url(url):
  html_str = requests.get(url).text
  return html_str

In [3]:
# Import Driver
browser = webdriver.Chrome()
browser.get(nba_url)

In [4]:
# Click the Moneyline Button to Check the Box
ml_button = browser.find_elements_by_xpath("//*[@id='__next']/div/main/div/div[3]/div[2]/div[1]/label[3]/input")[0]
ml_button.click()

# Click the Spread Button to Uncheck the Box
spread_button = browser.find_elements_by_xpath("//*[@id='__next']/div/main/div/div[3]/div[2]/div[1]/label[1]/input")[-1]
spread_button.click()


In [5]:
# Stores the current state of the browser into the variable.
html = browser.page_source

In [6]:
# Creates a list to store the scraped Odds.
odds = []

In [7]:
# Starts to Scrape using BeautifulSoup
soup = BeautifulSoup(html)
odds_soup = soup.find_all('div', class_='custom-1u3dgby e4ukmj14')

In [8]:
odds_soup

[<div class="custom-1u3dgby e4ukmj14" data-lazyload-listened="26"><table><thead><tr><th>Scheduled</th><th>Open</th><th class="custom-1ui3ly9 ebi37p0"><a class="" href="https://switchboard.actionnetwork.com/offers?bookId=68&amp;stateCode=world&amp;context=web-compareodds-banner&amp;deviceId=df513d83-d679-468b-9402-61c7e3a20374"><div class="compare-odds-book-header-cell__logo"><img alt="DraftKings" class="" height="21" src="https://assets.actionnetwork.com/477013_DraftKings@1x.png"/></div></a></th><th class="custom-1ui3ly9 ebi37p0"><a class="" href="https://switchboard.actionnetwork.com/offers?bookId=75&amp;stateCode=world&amp;context=web-compareodds-banner&amp;deviceId=df513d83-d679-468b-9402-61c7e3a20374"><div class="compare-odds-book-header-cell__logo"><img alt="BetMGM NJ" class="" height="21" src="https://assets.actionnetwork.com/779359_BetMGM800x200@1x.png"/></div></a></th><th class="custom-1ui3ly9 ebi37p0"><a class="" href="https://switchboard.actionnetwork.com/offers?bookId=69&amp

In [12]:
def nba_odds(html):
    
    odds = []
    
    # Finds all the TR tags in the soup_table
    soup_table = odds_soup[0].find_all('tr')
    
    # The list of the bookies
    bookie_list = ['Draft Kings', 'Bet MGM', 'Fanduel', 'PointsBet', 'Sugar House', 'Bet365', 'Unibet', 'Caesar' + "'s " + 'Sportsbook']
    
    # Sets the index to 1
    ind = 1
    
    for game in soup_table[1:]:
        # Create dictionaries for odds of each team
        team1_odds = dict()
        team2_odds = dict()
        
        # Grabs the div tag with the specified class
        team_names = game.find_all('div', class_='game-info__team')
        
        # Grabs the team names from the html
        name1 = team_names[0].find(class_='game-info__team--desktop').text
        name2 = team_names[1].find(class_='game-info__team--desktop').text
        
        # Sets the Team Names in the Dictionaries
        team1_odds['Team'] = name1
        team2_odds['Team'] = name2
        
        # Access the tag for game odds
        game_layer1 = game.find_all('td', class_='best-odds__last-type-cell')
        
        # Sets the index to 0
        index = 0
        
        # Loops through the odds by each bookie
        for bookie in game_layer1:
            odds_team1 = bookie.find_all('span', class_='highlight-text__children')[0].text
            odds_team2 = bookie.find_all('span', class_='highlight-text__children')[2].text
        
            # Assigns the Name to the Dictionary
            bookie = bookie_list[index]
            team1_odds[bookie] = odds_team1
            team2_odds[bookie] = odds_team2
            
            index += 1
            
        ind += 1
        
        # Appends the odds information into the odds list
        odds.append(team1_odds)
        odds.append(team2_odds)
        
    # Instantiates the DataFrame of Soccer Odds
    nba_odds_df = pd.DataFrame(odds)
    pd.set_option('display.max_columns', 500)

    # Sends the odds to an excel sheet in a specified folder (This folder can be changed to wherever you want)
    dt = datetime.now()
    writer = pd.ExcelWriter(r'/Users/sarahcasale/Documents/NBA Odds/Scraped Data/' + dt.strftime('%b-%d-%Y') + '_nba_data' + '.xlsx')
    nba_odds_df.to_excel(writer, sheet_name='NBA', index=False, na_rep='NaN')
    
    # Auto-adjust columns' width
    for column in nba_odds_df:
        column_width = max(nba_odds_df[column].astype(str).map(len).max(), len(column))
        col_idx = nba_odds_df.columns.get_loc(column)
        writer.sheets['NBA'].set_column(col_idx, col_idx, column_width)
    
    writer.save()
    
    return nba_odds_df

In [13]:
nba_odds_df = nba_odds(html)

In [14]:
nba_odds_df

,Team,Draft Kings,Bet MGM,Fanduel,PointsBet,Sugar House,Bet365,Unibet,Caesar's Sportsbook
0,Wizards,-155,-150,-154,-160,-148,-155,-148,-160
1,Cavaliers,+135,+125,+130,+135,+123,+135,+123,+140
2,Nets,-475,-500,-500,-470,-480,-450,-480,-475
3,Magic,+350,+375,+385,+350,+360,+350,+360,+360
4,Bucks,+145,+145,+144,+145,+135,+145,+135,+145
5,Knicks,-165,-175,-172,-170,-162,-165,-162,-170
6,Raptors,+130,+125,+126,+125,+116,+130,+116,+130
7,Celtics,-150,-150,-148,-145,-139,-150,-139,-150
8,Pistons,+130,+125,+124,+130,+128,+140,+128,+130
9,Rockets,-150,-150,-146,-150,-152,-160,-152,-150
